## This exploration tests [functorch](https://pytorch.org/blog/pytorch-1.11-released/)'s per example gradient computation capabilities.

 We know that clipping gradients in DP-SGD introduces bias. 
 The following code studies the impact of adding symmetric pre-clipping noise on parameter recovery, suggested in Section 5 of [this](https://arxiv.org/pdf/2006.15429.pdf) paper
 on a toy logistic regression model.
 To isolate the effect of pre-clipping noise, we don't add DP noise.


In [1]:
%load_ext tensorboard

%tensorboard --logdir runs/


In [6]:
import pandas as pd
import numpy as np
import shutil


import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as lreg

from tqdm import tqdm

from functorch import make_functional, vmap, grad
from functorch import vmap, grad_and_value
from functorch import make_functional

from torch.utils.tensorboard import SummaryWriter


default_seed = 1234

torch.manual_seed(default_seed)
np.random.seed(default_seed)
N = 20000


shutil.rmtree("runs/",ignore_errors=True) 


'''
True parameter we would like to recover.
'''
d =3

theta = np.array([-0.1,1.0,-3.0])        

def create_un_separable_dataset(N,d):
    ####
    ## Not guaranteed to be linearly seperable.
    ###
    def sigmoid(x):
        return 1.0/(1.0+np.exp(-x))   
    
    Sigma =datasets.make_spd_matrix(d,random_state = default_seed)
    inputs = np.random.multivariate_normal(np.ones(d), Sigma, int(N))
    labels = np.random.binomial(1.0, sigmoid(inputs.dot(theta)))
    return torch.Tensor(inputs),torch.Tensor(labels)



class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim,bias=False) ## Setting model intercept to 0.
        
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
    

inputs,labels = create_un_separable_dataset(N,d)

X_train, X_test, y_train,  y_test = train_test_split( inputs, labels, test_size=0.2, random_state=default_seed)

C = 0.1 ## clipping threshold
epochs = 20000
learning_rate = 0.01
N = X_train.shape[0]
batch_size = int(np.sqrt(N))
theta = torch.tensor(theta)

'''
The variable k controls the variance of the pre-clipping noise we are adding.
'''
for k in [0.1,0.5,1.0,2.0,5.0,10.,20.,100.]:
    
    model_lr  = LogisticRegression(d,1)    
    model_init = torch.zeros(d)

    with torch.no_grad():        
        for param in model_lr.parameters():
            param.copy_(model_init)

    criterion = torch.nn.BCELoss()


    optimizer_batch   = torch.optim.SGD(model_lr.parameters(), lr=learning_rate) 
    writer = SummaryWriter()

    one = torch.ones(1)

    def compute_norm_and_clip_grads(grad):
        return (grad*torch.minimum(one,C/grad.norm(2,dim=-1)))    
   
    def get_sample_grads(x,y,model):    
        def compute_loss_and_output(weights, x, obesered_y):
            predicted_y = func_model(weights, x).squeeze()
            loss = criterion(predicted_y, obesered_y)
            return loss, predicted_y

        func_model, weights = make_functional(model)
        grads_loss_output = grad_and_value(compute_loss_and_output, has_aux=True)    
        ## using vmap from functorch for fast per example gradient calculations.
        sample_grads, (sample_loss, output) = vmap(grads_loss_output, (None, 0, 0))(weights, x, y)   
        return sample_grads, (sample_loss, output)

    param_list = torch.zeros((epochs,d))
    
    for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
        optimizer_batch.zero_grad()        

        batch_indices=torch.randint(0,N, (batch_size,))
        sample_grads, (sample_loss, output)=get_sample_grads(X_train[batch_indices],y_train[batch_indices],model_lr)        
        writer.add_scalar("training loss:", sample_loss.mean(), epoch)
        for param in model_lr.parameters():
            weights = param
        
        writer.add_scalar("distance from true theta:",torch.linalg.norm(weights- theta,dim=-1),epoch)


        sample_grads= sample_grads[0]
        
        '''
        Adding zero mean Gaussian noise to the gradients.
        '''
        if k >0.0:
            sample_grads+= k*torch.randn((batch_size,1,d,))
            
        '''
        Clipping grads with vmap.
        '''
        clipped_grads = vmap(compute_norm_and_clip_grads)(sample_grads)    

        for name, param in model_lr.named_parameters():
            param.grad = clipped_grads.sum(0)
        optimizer_batch.step()    

        
    for param in model_lr.parameters():
        print("pre-clipping noise added",param)
    print(theta)


Training Epochs: 100%|███████████████████████████████████████████| 20000/20000 [00:25<00:00, 797.80it/s]


pre-clipping noise added Parameter containing:
tensor([[-0.4662,  2.1820, -6.7592]], requires_grad=True)
tensor([-0.1000,  1.0000, -3.0000], dtype=torch.float64)


Training Epochs: 100%|███████████████████████████████████████████| 20000/20000 [00:27<00:00, 731.81it/s]


pre-clipping noise added Parameter containing:
tensor([[-0.2526,  1.3531, -4.0759]], requires_grad=True)
tensor([-0.1000,  1.0000, -3.0000], dtype=torch.float64)


Training Epochs: 100%|███████████████████████████████████████████| 20000/20000 [00:28<00:00, 693.87it/s]


...

pre-clipping noise added Parameter containing:
tensor([[-0.1372,  0.9170, -2.8401]], requires_grad=True)
tensor([-0.1000,  1.0000, -3.0000], dtype=torch.float64)


Training Epochs: 100%|███████████████████████████████████████████| 20000/20000 [00:31<00:00, 642.89it/s]


pre-clipping noise added Parameter containing:
tensor([[-0.1209,  0.7847, -2.2054]], requires_grad=True)
tensor([-0.1000,  1.0000, -3.0000], dtype=torch.float64)


Training Epochs: 100%|███████████████████████████████████████████| 20000/20000 [00:31<00:00, 643.33it/s]

pre-clipping noise added Parameter containing:
tensor([[ 0.2249,  0.1294, -1.3428]], requires_grad=True)
tensor([-0.1000,  1.0000, -3.0000], dtype=torch.float64)


In [7]:
%load_ext tensorboard

%tensorboard --logdir runs/


## In the tensorboard, click on the "Wall" axis to compare the training loss and distance of the recovered parameter from the true theta.
## We observe that increasing k reduce the training error and improves the accuracy of estimated parameter initially. On the other hand, too large noise scales produce the opposite effect. 